# HL7-V2


`Important points:`
- (`Text based`)
- <u>Structure:</u>
    -  ` Trigger Event` - It is the event that takes place which interm generate the message
    -   `Segments`: data-field delimeter seperated e.g. <cr>, Identified by three leter code eg. MHS, PID, etc.,
        - `Data-fields`: these are the data between the data-field seperator (|).  e.g. |A01|20150601135823+0100| Datafields can also repeat with the data field seperator ("|") using the repetition seperator character ('~').
            -   `components`: they are the data inside a data-field seperated by componet-seperator delimeter (^). e.g. |23432^Smith^Gordon^Denny^Jr^MD^^^AssignAuth&1.2.3.4.5.6&ISO^L^9^1000^DN^ AssignFac&1.2.3.4.5.7&ISO^^G^20100101000000+0100^20330101000000+0100^doctor|
                -   `sub-components`: they are the parts of the componets and they are seperated by sub-compoment seperator delimeter (&) if Present.
                                     e.g. ^AssignFac&1.2.3.4.5.7&ISO^


for reference use: https://hl7-definition.caristix.com/v2/HL7v2.5.1/

- ### Example of `HL7-v2.5` message

- enter the source file path

In [ ]:
input_file_name='hl7v2_test'  #replace with your hl7v2.5 file path

-   reading the files segment wise

In [ ]:
messageSegments=[]
with open(input_file_name+".txt") as fd:
    messageSegments=fd.readlines() #readline will read the entire file line by line i.e. segment by segment

- first segment is the message header

In [ ]:
msg_header=''
if messageSegments and messageSegments[0].startswith("MSH"):
    msg_header=messageSegments[0]
else: 
    raise Exception("Message Header Not Found")

In [ ]:
from util.encoding import get_encoding_characters
message_encoding_chars=get_encoding_characters(
    messageHeaderSegment=msg_header
)

- 4th character in the message header is the field delimeter

In [ ]:
dataFieldDelimeter=msg_header[3]

- create the list of datafields for each segment

In [ ]:
SegmentWiseDataFieldLists=[]
for segment in messageSegments:
    dataFieldSegmentList=segment.split(message_encoding_chars['dataFieldSep'])
    SegmentWiseDataFieldLists.append(dataFieldSegmentList)

In [ ]:
print("message Header dataFieldSegments:: ",SegmentWiseDataFieldLists[0])

In [ ]:
safeStringPatern='********************************************************************************************************************************************************************************************************'

In [ ]:
message_structure_dict={}
for segment in messageSegments:
    segment=segment.replace(f"{message_encoding_chars['escapeSeq']}{message_encoding_chars['dataFieldSep']}",safeStringPatern)
    dataFieldsList=segment.split(message_encoding_chars['dataFieldSep'])
    segment_key=dataFieldsList[0]
    temp_data_dict={}
    if segment_key=="MSH":
        temp_data_dict["MSH.1"]=message_encoding_chars['dataFieldSep']
    for data_key, dataField in enumerate(dataFieldsList[1:]):
        data_level_key=f"{segment_key}.{data_key+2 if segment_key=='MSH' else data_key+1}"
        dataField=dataField.replace(safeStringPatern, f"{message_encoding_chars['escapeSeq']}{message_encoding_chars['dataFieldSep']}")

        if data_level_key!="MSH.2":
            dataField=dataField.replace(f"{message_encoding_chars['escapeSeq']}{message_encoding_chars['repetitionSep']}",safeStringPatern)
            dataFieldList=dataField.split(message_encoding_chars['repetitionSep'])       
            tempDataFieldList=[]
            for repeat_key,data in enumerate(dataFieldList):
                data=data.replace(safeStringPatern,f"{message_encoding_chars['escapeSeq']}{message_encoding_chars['repetitionSep']}")
                data=data.replace(f"{message_encoding_chars['escapeSeq']}{message_encoding_chars['componentSep']}",safeStringPatern)
                componentList=data.split(message_encoding_chars['componentSep'])
                temp_component_dict={}
                if len(componentList)>1:
                    for component_key, component in enumerate(componentList):
                        component=component.replace(safeStringPatern,f"{message_encoding_chars['escapeSeq']}{message_encoding_chars['componentSep']}")
                        component=component.replace(f"{message_encoding_chars['escapeSeq']}{message_encoding_chars['sub-componentSep']}",safeStringPatern)
                        subComponentList=component.split(message_encoding_chars['sub-componentSep'])
                        component_level_key=data_level_key+f".{component_key+1}"
                        temp_subcomponent_dict={}
                        if len(subComponentList)>1:
                            for subcompnent_key, subcompnent in enumerate(subComponentList):
                                subcompnent=subcompnent.replace(safeStringPatern,f"{message_encoding_chars['escapeSeq']}{message_encoding_chars['sub-componentSep']}")
                                subcompnent_level_key=component_level_key+f".{subcompnent_key+1}"

                                temp_subcomponent_dict[subcompnent_level_key]=subcompnent
                        
                        temp_component_dict[component_level_key]=temp_subcomponent_dict if len(subComponentList)>1 else component
            
                tempDataFieldList.append(temp_component_dict if len(componentList)>1 else data.replace(safeStringPatern,f"{message_encoding_chars['escapeSeq']}{message_encoding_chars['componentSep']}"))
            temp_data_dict[data_level_key]=tempDataFieldList if len(tempDataFieldList)>1 else tempDataFieldList[0]
        else:
            temp_data_dict[data_level_key]=dataField
        
    message_structure_dict[segment_key]=temp_data_dict if temp_data_dict else None




from datetime import datetime 
import json,os
res:int
output_dir='./output_folder'
output_file_name=f"{input_file_name}_output_{datetime.now().date()}.json"
output_file_path=os.path.join(output_dir, output_file_name)
with open(output_file_path, '+w') as fd:
    res=fd.write(json.dumps(message_structure_dict,indent=4))
    print("Data write successful :)\nOutput file name:: ",output_file_path)
